In [ ]:
from dotenv import load_dotenv
import os
import re
import os
import pandas as pd
import numpy as np
from groq import Groq
from stockfish import Stockfish

In [ ]:
load_dotenv()  # Load variables from .env
STOCKFISH_PATH=os.environ.get("STOCKFISH_PATH")
LLAMA_MODEL_PATH=os.environ.get("LLAMA_MODEL_PATH")
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
stockfish = Stockfish(STOCKFISH_PATH,depth=20,parameters={"Threads": 10,"Minimum Thinking Time": 50})
d1=pd.read_csv('theory/a.tsv', sep='\t', header=0)
d2=pd.read_csv('theory/b.tsv', sep='\t', header=0)
d3=pd.read_csv('theory/c.tsv', sep='\t', header=0)
d4=pd.read_csv('theory/d.tsv', sep='\t', header=0)
d5=pd.read_csv('theory/e.tsv', sep='\t', header=0)

In [ ]:
def clean(response):
    response=re.sub(r'<think>.*?</think>','',response,flags=re.DOTALL)
    return response

In [ ]:
def get_best_move(fen):
    stockfish.set_fen_position(fen)
    return stockfish.get_best_move()

In [ ]:
def ch_comp_bm_w_exp(fen,best_move,type_of_move):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a chess coach , explain it well as if you thought of the move and try to focus more on the move and why it is the best move, be extremely nice to the student",
            },
            {
                "role": "user",
                "content": f"Justify why {type_of_move} {best_move} is the best move for the FEN {fen}",
            }
        ],
        model="deepseek-r1-distill-llama-70b",
    )

    return clean(chat_completion.choices[0].message.content)

In [40]:
def ch_comp_th(eco,name,pgn):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a chess coach , explain a theory behind the opening of the move with the name {name} ,ECO {eco} and the PGN {pgn} , please tell ALL THE pros and cons of the opening . All the variations and how to play against them.",
            },
            {
                "role": "user",
                "content": f"Explain the pros and cons and how to play against the best variations of the opening of the name {name} ,ECO {eco} and the PGN {pgn}",
            }
        ],
        model="deepseek-r1-distill-llama-70b",
    )

    return clean(chat_completion.choices[0].message.content)

In [42]:
def find(theory):
    for x in range(0,len(d1["name"])):
        if d1["name"][x].lower()==theory.lower():
            return d1['eco'][x],d1['name'][x],d1['pgn'][x]
    for x in range(0,len(d2["name"])):
        if d2["name"][x].lower()==theory.lower():
            return d2['eco'][x],d2['name'][x],d2['pgn'][x]
    for x in range(0,len(d3["name"])):
        if d3["name"][x].lower()==theory.lower():
            return d3['eco'][x],d3['name'][x],d3['pgn'][x]

In [ ]:
def th_w_exp():
    theory=input("You : ")
    eco,name,pgn=find(theory)
    return ch_comp_th(eco,name,pgn)

In [ ]:
def bm_w_exp():
    fen = input("You : ")
    best_move = get_best_move(fen)
    g = str(stockfish.will_move_be_a_capture(best_move)).split(".")[1]
    if g=="NO_CAPTURE":
        g=False
    else:
        g=True
    p1_split = str(stockfish.get_what_is_on_square(best_move[:2])).split(".")
    p2_split = str(stockfish.get_what_is_on_square(best_move[2:])).split(".")
    p1 = p1_split[1] if len(p1_split) > 1 else p1_split[0]
    p2 = p2_split[1] if len(p2_split) > 1 else p2_split[0]
    if g:
        type_of_move = f"Capturing the {p2} with {p1}"
    elif "e8g8" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e8b8" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e1g1" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e1b1" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    else:
        type_of_move = "Move"
    print("Bot : ", ch_comp_bm_w_exp(fen, best_move, type_of_move))

In [41]:
print(th_w_exp())



The French Defense, starting with 1. e4 e6, is a dynamic and strategic opening that offers both opportunities and challenges for Black. Here's a structured overview of its pros, cons, variations, and strategies:

### Pros of the French Defense:
1. **Central Challenge**: By playing d5, Black challenges White's central control, potentially weakening White's position.
2. **Solid Structure**: The French Defense is known for its resilience, making it difficult for White to break through.
3. **Dynamic Positions**: Variations like the Winawer can lead to complex and dynamic positions, offering counterplay on the queenside.

### Cons of the French Defense:
1. **Passive Position**: After the Exchange Variation, Black may end up with a passive position and an isolated pawn on d5.
2. **Cramped Position**: The French can lead to a cramped setup for Black, with potential issues developing the light-squared bishop.

### Main Variations:
1. **Exchange Variation (3. exd5)**: White gains central cont